In [2]:
!pip install stable_baselines3
!pip install gymnasium
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
!pip install yfinance


[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip


  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\11429\appdata\local\temp\pip-req-build-adqdqw85
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 453349afc2ee5eac8cc7f5766638c50f595df2d0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\11429\appdata\local\temp\pip-install-yy_221_c\elegantrl_2b1aba6f51174d65b200bde56ef1c73c
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit b4b9d662b9f9cb7cc368ac2b1036b5119eb20be4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached exchange_calendars-4.2.8-py3-none-an

  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\11429\AppData\Local\Temp\pip-req-build-adqdqw85'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\11429\AppData\Local\Temp\pip-install-yy_221_c\elegantrl_2b1aba6f51174d65b200bde56ef1c73c'
ERROR: Package 'finrl' requires a different Python: 3.8.3 not in '<3.11,>=3.10'

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: C:\Python38\python.exe -m pip install --upgrade pip


In [3]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from stable_baselines3 import A2C, PPO, DDPG
from stockEnv import StockEnvMine
from finrl.meta.preprocessor.preprocessors import data_split
from Evaluation import RetrieveBenchmark

In [4]:
df = pd.read_csv("Pre_ProcessdFile.csv")

In [5]:
dic = {"ppo": PPO, "a2c": A2C, "ddpg": DDPG}
lis = ["a2c", "a2c", "ddpg", "a2c", "a2c", "ddpg"]
models = []
for i in lis:
    model = dic[i].load("model_{}".format(lis.index(i) + 1))
    models.append(model)

c:\Users\11429\anaconda3\envs\Pytorch\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_make_function' on <module 'cloudpickle.cloudpickle' from 'c:\\Users\\11429\\anaconda3\\envs\\Pytorch\\lib\\site-packages\\cloudpickle\\cloudpickle.py'>
  warnings.warn(


In [6]:
INDICATORS = ['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [7]:
last_state = None
initial = True
deterministic = True
account_list = []
actions_list = []
val_start = ['2022-04-04', '2022-07-06', '2022-10-04', '2023-01-04', '2023-04-05', '2023-07-07']
val_end = ['2022-07-06', '2022-10-04', '2023-01-04', '2023-04-05', '2023-07-07', '2023-08-30']
stock_dimension = len(df.tic.unique())
state_space = 1 + 2*stock_dimension + 8 *stock_dimension
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
InitialAmount = 1000000
reward_scaling = 1e-4
env_kwargs = {
    "hmax": 100,
    "initial_amount": InitialAmount,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": reward_scaling
}
print("""There are {} stocks in the dataset.
The state space we used for training is {}.
The indicators we used is {}.
The reward scaling is {}.
The initial amount is {}.
The share for each storck in the beginning is 0.
Cost of buying and selling is 0.001.
    """.format(stock_dimension, state_space, INDICATORS, reward_scaling, InitialAmount))

There are 29 stocks in the dataset.
The state space we used for training is 291.
The indicators we used is ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma'].
The reward scaling is 0.0001.
The initial amount is 1000000.
The share for each storck in the beginning is 0.
Cost of buying and selling is 0.001.
    


In [8]:
for i in range(len(models)):
    trade_data = data_split(df, start=val_start[i], end=val_end[i],)
    trade_env = StockEnvMine(df=trade_data, turbulence_th=None, iteration=iter, mode="", model_name="", last_state=last_state, initial=initial, **env_kwargs)
    env, obs = trade_env.getDummyEnv()
    account_memory = None
    actions_memory = None
    env.reset()
    max_step = len(trade_env.df.index.unique()) - 1
    for j in range(max_step + 1):
        action, states = models[i].predict(obs, deterministic=deterministic)
        obs, rewards, dones, info = env.step(action)
        if j == max_step - 1:
            account_memory = env.env_method(method_name="saveAssetMemory")
            actions_memory = env.env_method(method_name="saveActionMemory")
            last_state = env.envs[0].render()
        if dones[0]:
            print("Finished")
            break
    if i == 0:
        initial = False
    account_list.append(account_memory[0])
    actions_list.append(actions_memory[0])
a_new = pd.concat(account_list, ignore_index=True)

day: 62, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 953728.16
total_reward: -46271.84
total_cost: 1062518.92
total_trades: 837
Sharpe: -0.814
Finished
day: 62, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 942405.33
total_reward: -57594.67
total_cost: 0.00
total_trades: 851
Sharpe: -1.160
Finished
day: 62, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1138725.23
total_reward: 138725.23
total_cost: 495160.37
total_trades: 1074
Sharpe: 2.952
Finished
day: 62, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1132177.92
total_reward: 132177.92
total_cost: 347303.62
total_trades: 844
Sharpe: 1.623
Finished
day: 62, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1120217.12
total_reward: 120217.12
total_cost: 0.00
total_trades: 806
Sharpe: 1.592
Finished
day: 37, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1184481.30
total_reward: 184481.30
total_cost: 1308.40
total_trades: 630
Sharpe: 3.684
Finished


In [9]:
a_new.set_index('date', inplace=True)
a_new.index = pd.to_datetime(a_new.index)
dji = RetrieveBenchmark()

[*********************100%%**********************]  1 of 1 completed


In [10]:
plt.figure()

# Plot each DataFrame
plt.plot(dji['adjusted_value'], label='DJIA(Benchmark)',linewidth=4.0)
plt.plot(a['account_value'], label='Ensembled Agent',linewidth=4.0)

# Adding titles and labels
plt.title('Comparison of Account Balance')
plt.xlabel('Date')
plt.ylabel('Account Balance')

# Add a legend
plt.legend()
plt.grid()
# Show the plot
plt.show()

: 